# TAP v. Butler

A race to find a certain Object when we know its Id, patch, and tract.

In [1]:
from lsst.rsp import get_tap_service, retrieve_query
import lsst.daf.butler as dafButler
import numpy as np

In [2]:
mytract = 4431
mypatch = 17
myobjectid = 1909948454470206951

## Butler

In [3]:
butler = dafButler.Butler('dp02', collections='2.2i/runs/DP0.2')

In [4]:
%%time
datasetType = 'objectTable'
dataId = {'tract': mytract, 'patch': mypatch}
obj = butler.get(datasetType, dataId=dataId)

CPU times: user 962 ms, sys: 745 ms, total: 1.71 s
Wall time: 1.39 s


In [5]:
# obj

In [6]:
print(f"Retrieved catalog of {len(obj)} objects.")

Retrieved catalog of 37026 objects.


In [7]:
%%time
ids = obj.index.to_numpy()
ras = obj.coord_ra.to_numpy()
decs = obj.coord_dec.to_numpy()
tx = np.where( ids == myobjectid )[0]
print('FOUND:', tx, ids[tx], ras[tx], decs[tx])

[37021] [55.65390937] [-32.19286897]
CPU times: user 1.16 ms, sys: 52 µs, total: 1.21 ms
Wall time: 1.05 ms


In [8]:
del ids, ras, decs, tx
del datasetType, dataId, obj, butler

## TAP

In [9]:
service = get_tap_service()

Just start with tract and patch, like we do with the Butler.

TOO SLOW

In [10]:
# %%time
# query = "SELECT ObjectId " + \
#         "FROM dp02_dc2_catalogs.Object " + \
#         "WHERE tract = "+str(mytract)+" AND patch = "+str(mypatch)+" "
# results = service.search(query)

In [11]:
# table = results.to_table()
# print(f"Retrieved catalog of {len(table)} objects.")

In [12]:
# table

In [13]:
# %%time
# tx = np.where(table['ObjectId'] == myobjectid)[0]
# print(tx[0], table['ObjectId'][tx[0]])

In [14]:
# del query, results, table, tx

Now forget about knowing tract and patch, and query only on the known objectId

TOO SLOW

In [15]:
# %%time
# query = "SELECT ObjectId " + \
#         "FROM dp02_dc2_catalogs.Object " + \
#         "WHERE ObjectId = "+str(myobjectid)+" "
# results = service.search(query)

In [16]:
# table = results.to_table()
# print(f"Retrieved catalog of {len(table)} objects.")

In [17]:
# table

In [18]:
# del query, results, table

Now forget about knowing tract and patch, and use the RA Dec.

In [19]:
%%time
query = "SELECT ObjectId " + \
        "FROM dp02_dc2_catalogs.Object " + \
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec ), " + \
        "CIRCLE('ICRS', 55.6539, -32.19286, 0.01)) = 1 "
results = service.search(query, maxrec=1000)

CPU times: user 13.5 ms, sys: 34.7 ms, total: 48.2 ms
Wall time: 3.5 s


In [20]:
table = results.to_table()
print(f"Retrieved catalog of {len(table)} objects.")

Retrieved catalog of 215 objects.


In [21]:
# table

In [22]:
%%time
tx = np.where(table['ObjectId'] == myobjectid)[0]
print('FOUNDtx[0], table['ObjectId'][tx[0]])

151 1909948454470206951
CPU times: user 358 µs, sys: 147 µs, total: 505 µs
Wall time: 447 µs


In [23]:
del query, results, table